In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv("/kaggle/input/petfinder-pawpularity-score/train.csv")
test_df = pd.read_csv("/kaggle/input/petfinder-pawpularity-score/test.csv")

In [ ]:
import pandas as pd
import numpy as np
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import LSTM, Dropout, Conv2D, TimeDistributed, MaxPooling2D, Flatten, Activation
from keras.layers import Dense

In [ ]:
train_df['Id'] = train_df['Id']+'.jpg'
train_df["Pawpularity"] = (train_df["Pawpularity"]-1)/(100-1)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train, x_valid = train_test_split( train_df, test_size=0.2, random_state=42, stratify = train_df["Pawpularity"])

In [ ]:
from keras_preprocessing.image import ImageDataGenerator
DataGen = ImageDataGenerator( rescale=1./255., shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
train_generator=DataGen.flow_from_dataframe(dataframe=X_train, 
                                            directory='/kaggle/input/petfinder-pawpularity-score/train', 
                                            x_col="Id", y_col="Pawpularity" ,
                                            class_mode='raw',
                                            target_size=(299,299), batch_size=16)

valid_generator=DataGen.flow_from_dataframe(dataframe=x_valid, 
                                            directory='/kaggle/input/petfinder-pawpularity-score/train', 
                                            x_col="Id", y_col="Pawpularity" ,
                                            class_mode='raw', 
                                            target_size=(299, 299), batch_size=16)

In [ ]:
# DataGen = ImageDataGenerator( rescale=1./255., shear_range=0.2,
#         zoom_range=0.2,
#         horizontal_flip=True)
# train_generator1=DataGen.flow_from_dataframe(dataframe=X_train, 
#                                             directory='/kaggle/input/petfinder-pawpularity-score/train', 
#                                             x_col="Id", y_col="Pawpularity" ,
#                                             class_mode='raw',
#                                             target_size=(224,224), batch_size=16)

# valid_generator1=DataGen.flow_from_dataframe(dataframe=x_valid, 
#                                             directory='/kaggle/input/petfinder-pawpularity-score/train', 
#                                             x_col="Id", y_col="Pawpularity" ,
#                                             class_mode='raw', 
#                                             target_size=(224,224), batch_size=16)

In [ ]:
import tensorflow as tf
base_model = tf.keras.applications.InceptionResNetV2(include_top=True, weights=None,input_shape = (299,299,3))

In [ ]:
# base_model1 = tf.keras.applications.ResNet50(include_top=True, weights=None,input_shape = (224,224,3))

In [ ]:
x = base_model.layers[-2].output
x= Dense(252, activation='relu')(x)
x= Dense(126, activation='relu')(x)
x= Dense(1, activation='sigmoid')(x)
model = Model(inputs = base_model.input, outputs =x)

In [ ]:
# x1 = base_model1.layers[-2].output
# x1= Dense(252, activation='relu')(x1)
# x1= Dense(126, activation='relu')(x1)
# x1= Dense(1, activation='sigmoid')(x1)
# model1 = Model(inputs = base_model1.input, outputs =x1)

In [ ]:
model.compile(tf.keras.optimizers.Adam(learning_rate=0.0001),loss="mse")
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

In [ ]:
# model1.compile(tf.keras.optimizers.Adam(learning_rate=0.0001),loss="mse")
# STEP_SIZE_TRAIN1=train_generator1.n//train_generator1.batch_size
# STEP_SIZE_VALID1=valid_generator1.n//valid_generator1.batch_size

In [ ]:
Earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
reg = model.fit(train_generator, 
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=90,callbacks=[Earlystop])

In [ ]:
# reg1 = model1.fit(train_generator1, 
#                     steps_per_epoch=STEP_SIZE_TRAIN1,
#                     validation_data=valid_generator1,
#                     validation_steps=STEP_SIZE_VALID1,
#                     epochs=90,callbacks=[Earlystop])

In [ ]:
test_df['Id'] = test_df['Id']+'.jpg'
test_df

In [ ]:
test_datagen=ImageDataGenerator(rescale=1./255.)
test_generator=test_datagen.flow_from_dataframe(
dataframe=test_df,
directory='/kaggle/input/petfinder-pawpularity-score/test',
x_col="Id",
y_col=None,
batch_size=1,
seed=42,
shuffle=False,
class_mode=None,
target_size=(299,299))

In [ ]:
# test_datagen=ImageDataGenerator(rescale=1./255.)
# test_generator1=test_datagen.flow_from_dataframe(
# dataframe=test_df,
# directory='/kaggle/input/petfinder-pawpularity-score/test',
# x_col="Id",
# y_col=None,
# batch_size=1,
# seed=42,
# shuffle=False,
# class_mode=None,
# target_size=(224,224))

In [ ]:
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
pred=model.predict(test_generator, steps=STEP_SIZE_TEST, verbose=1)

In [ ]:
# STEP_SIZE_TEST1=test_generator1.n//test_generator1.batch_size
# pred1=model1.predict(test_generator1, steps=STEP_SIZE_TEST1, verbose=1)

In [ ]:
final_pred = pred *(100-1) + 1

In [ ]:
# final_pred1 = pred1 *(100-1) + 1

In [ ]:
# final = (final_pred+final_pred1)/2

In [ ]:
test_df = pd.read_csv("/kaggle/input/petfinder-pawpularity-score/test.csv")
final_submission=pd.DataFrame()
final_submission['Id'] = test_df['Id']
final_submission['Pawpularity'] = pd.DataFrame(final_pred)[0]

In [ ]:
final_submission

In [ ]:
final_submission.to_csv("submission.csv",index = False)

In [ ]:
# mse...
# To do :
#     Experimentation and Fine tuning of pretrained models
#     Inclusion of tabular info